In [1]:
import pandas as pd
import numpy as np

import nltk
#from bs4 import BeautifulSoup

import gensim
from gensim import utils
import sys
from sklearn.datasets import fetch_20newsgroups
from nltk import word_tokenize
from nltk import download
from nltk.corpus import stopwords
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity

from os import listdir
from os.path import isfile, join

import json

%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
from gensim.models.keyedvectors import KeyedVectors

model = KeyedVectors.load_word2vec_format('../WWW2018/words/GoogleNews-vectors-negative300.bin.gz', binary=True)
#model.save_word2vec_format('./words/GoogleNews-vectors-negative300.txt', binary=False)

In [3]:
download('punkt') #tokenizer, run once
download('stopwords') #stopwords dictionary, run once
stop_words = stopwords.words('english')

[nltk_data] Downloading package punkt to /Users/wjkim/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/wjkim/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
instagram_posts = pd.read_csv('./instagram/instagram_buzzfeed.csv')
instagram_posts

,Publisher,Instagram ID,URL,Type,comment
0,ABC News Politics,abcnews,https://instagram.com/p/BK3CwIohg-c/,photo,Catherine Duchess of Cambridge and Prince Will...
1,ABC News Politics,abcnews,https://instagram.com/p/BK187WNhOim/,video,The Presidential Debate ends on a handshake be...
2,ABC News Politics,abcnews,https://instagram.com/p/BK15huIBVkb/,photo,Donald Trump speaks during the Presidential De...
3,ABC News Politics,abcnews,https://instagram.com/p/BK14VF8hBjH/,photo,Former U.S. President Bill Clinton talks with ...
4,ABC News Politics,abcnews,https://instagram.com/p/BK10vR3hqj2/,photo,Republican nominee Donald Trump looks on durin...
5,ABC News Politics,abcnews,https://instagram.com/p/BK10liYB_1W/,photo,Hillary Clinton speaks during the #Presidentia...
6,ABC News Politics,abcnews,https://instagram.com/p/BK1zj1NhB-y/,photo,Hillary Clinton shakes hands with Donald Trump...
7,ABC News Politics,abcnews,https://instagram.com/p/BK1yleuBmqS/,photo,Melania Trump shakes hands with Bill Clinton b...
8,ABC News Politics,abcnews,https://instagram.com/p/BK1ybLZB-9S/,video,Presidential nominees Hillary Clinton and Dona...
9,ABC News Politics,abcnews,https://instagram.com/p/BK1q3OvBLbe/,photo,Miami Marlins players wearing No. 16 tonight i...


In [18]:
instagram_text_list = instagram_posts['comment']
instagram_text_list

0      Catherine Duchess of Cambridge and Prince Will...
1      The Presidential Debate ends on a handshake be...
2      Donald Trump speaks during the Presidential De...
3      Former U.S. President Bill Clinton talks with ...
4      Republican nominee Donald Trump looks on durin...
5      Hillary Clinton speaks during the #Presidentia...
6      Hillary Clinton shakes hands with Donald Trump...
7      Melania Trump shakes hands with Bill Clinton b...
8      Presidential nominees Hillary Clinton and Dona...
9      Miami Marlins players wearing No. 16 tonight i...
10     TONIGHT: Hillary Clinton and Donald Trump take...
11     Surf dog Skyler who is a Red Australian Shephe...
12     1962 photo shows golf legend Arnold Palmer lif...
13     JUST IN: Arnold Palmer one of the greatest and...
14     Major League Baseball teams across the country...
15     Former President George W. Bush receives a hug...
16     Fitness trainer Deanna Jefferson and her fianc...
17     Stunning video shows dee

In [5]:
facebook_posts = {}
facebook_posts_dir = './facebook_json'
for f in listdir(facebook_posts_dir):
    full_path = join(facebook_posts_dir, f)
    
    if full_path != './facebook_json/.DS_Store':
        #print(full_path)
        fb_json = json.load(open(full_path))
        print(f)
        #facebook_posts.append(fb_json)
        if 'message' in fb_json:
            facebook_posts[f] = fb_json['message']
        else:
            facebook_posts[f] = ''
        #pprint(data['id'])
    
    #if isfile(full_path):
        #print(join(facebook_img_dir, f))
        #facebook_posts.append(join(facebook_posts_dir, f))

1784_62317591679_10153861481676680.json
2280_114517875225866_1472557792755194.json
0354_219367258105115_1281841091857721.json
1752_62317591679_10153861035666680.json
0705_219367258105115_1291002827608214.json
1975_389658314427637_1535093403217450.json
1555_62317591679_10153847662691680.json
1294_346937065399354_1252978291461889.json
1008_135665053303678_554505034753009.json
0526_219367258105115_1285349598173537.json
0443_219367258105115_1283655648342932.json
1209_346937065399354_1247777691981949.json
0889_135665053303678_552599614943551.json
0004_184096565021911_1035352946562931.json
1718_62317591679_10153859749961680.json
1553_62317591679_10153847645156680.json
2225_114517875225866_1466136353397338.json
0825_135665053303678_552041008332745.json
1908_389658314427637_1533231846736939.json
1212_346937065399354_1247825678643817.json
1399_62317591679_10153842037026680.json
1077_440106476051475_1248049411923840.json
0602_219367258105115_1286887648019732.json
0005_184096565021911_10353665798

1722_62317591679_10153859901176680.json
1401_62317591679_10153842058606680.json
0096_184096565021911_1038031976295028.json
0114_184096565021911_1038804176217808.json
1605_62317591679_10153849837396680.json
0335_146422995398181_1457100920997042.json
0868_135665053303678_552451568291689.json
0681_219367258105115_1290632657645231.json
0773_135665053303678_551518585051654.json
047_184096565021911_1036399716458254.json
0659_219367258105115_1290363481005482.json
0544_219367258105115_1285711234804040.json
1850_62317591679_10153863046216680.json
1972_389658314427637_1534794629913994.json
2205_114517875225866_1464796833531290.json
0673_219367258105115_1290530250988805.json
1979_389658314427637_1535160086544115.json
0945_135665053303678_553105428226303.json
1122_440106476051475_1252426128152835.json
2168_114517875225866_1462710973739876.json
0073_184096565021911_1037288223036070.json
1319_346937065399354_1253151444777907.json
1296_346937065399354_1252992964793755.json
1594_62317591679_1015384933

1323_346937065399354_1253224401437278.json
1019_135665053303678_554510764752436.json
1029_135665053303678_554703414733171.json
1312_346937065399354_1252672244825827.json
1480_62317591679_10153845003471680.json
0768_135665053303678_551518305051682.json
1708_62317591679_10153859478516680.json
2011_389658314427637_1535745026485621.json
2103_389658314427637_1540351249358332.json
1196_346937065399354_1247104542049264.json
1692_62317591679_10153853200791680.json
0435_219367258105115_1283507681691062.json
1477_62317591679_10153844940006680.json
0775_135665053303678_551518988384947.json
0522_219367258105115_1285323381509492.json
0450_219367258105115_1283726711669159.json
1864_62317591679_10153863548616680.json
0175_184096565021911_1041765635921662.json
0425_219367258105115_1283398388368658.json
2111_389658314427637_1540494449344012.json
0122_184096565021911_1038973039534255.json
1694_62317591679_10153853252201680.json
0189_184096565021911_1042112815886944.json
0473_219367258105115_128438451493

1497_62317591679_10153845862231680.json
1974_389658314427637_1534822329911224.json
1215_346937065399354_1247926135300438.json
1368_62317591679_10153841027911680.json
0033_184096565021911_1035825603182332.json
1465_62317591679_10153844548621680.json
0646_219367258105115_1290186997689797.json
1282_346937065399354_1252876628138722.json
0106_184096565021911_1038577226240503.json
1677_62317591679_10153852595686680.json
1527_62317591679_10153846817696680.json
1664_62317591679_10153852153391680.json
1980_389658314427637_1535192106540913.json
2038_389658314427637_1536289316431192.json
1218_346937065399354_1248417465251305.json
0486_219367258105115_1284556298252867.json
1108_440106476051475_1249132908482157.json
1395_62317591679_10153841929731680.json
1700_62317591679_10153859022796680.json
2208_114517875225866_1464970846847222.json
0621_219367258105115_1289642367744260.json
1541_62317591679_10153847279056680.json
1856_62317591679_10153863282756680.json
0735_219367258105115_1291661910875639.jso

1911_389658314427637_1533342506725873.json
0893_135665053303678_552599881610191.json
1726_62317591679_10153860062136680.json
0744_219367258105115_1291874880854342.json
0225_146422995398181_1447950298578771.json
2210_114517875225866_1465022330175407.json
1193_346937065399354_1246907302068988.json
1633_62317591679_10153850782621680.json
0466_219367258105115_1284230444952119.json
2018_389658314427637_1536113469782110.json
0798_135665053303678_551828481687331.json
0765_135665053303678_551518175051695.json
1495_62317591679_10153845627766680.json
1168_346937065399354_1245902942169424.json
0234_146422995398181_1448516865188781.json
1476_62317591679_10153844918141680.json
1327_346937065399354_1253262431433475.json
1500_62317591679_10153845915901680.json
1963_389658314427637_1534123166647807.json
1873_62317591679_10153863798951680.json
1797_62317591679_10153861566101680.json
0974_135665053303678_554123541457825.json
0220_146422995398181_1447689015271566.json
1162_346937065399354_124586785550626

2257_114517875225866_1471179969559643.json
1185_346937065399354_1246675775425474.json
0570_219367258105115_1286155644759599.json
1457_62317591679_10153844234911680.json
0790_135665053303678_551475151722664.json
1581_62317591679_10153848797731680.json
2178_114517875225866_1463586530318987.json
1420_62317591679_10153842861371680.json
1322_346937065399354_1253200038106381.json
0758_135665053303678_551473965056116.json
1937_389658314427637_1533775493349241.json
0273_146422995398181_1450910671616067.json
0780_135665053303678_551599835043529.json
0772_135665053303678_551518525051660.json
1909_389658314427637_1533245070068950.json
0890_135665053303678_552599668276879.json
1685_62317591679_10153852915751680.json
2270_114517875225866_1472225646121742.json
0228_146422995398181_1447962805244187.json
1550_62317591679_10153847550156680.json
1751_62317591679_10153861001506680.json
2251_114517875225866_1470917416252565.json
2250_114517875225866_1470915582919415.json
1264_346937065399354_1250119471747

2223_114517875225866_1466120426732264.json
1668_62317591679_10153852331841680.json
0715_219367258105115_1291273510914479.json
1253_346937065399354_1249822718444113.json
0750_135665053303678_551096721760507.json
1769_62317591679_10153861397561680.json
1095_440106476051475_1248905528504895.json
0032_184096565021911_1035801043184788.json
0720_219367258105115_1291404070901423.json
0077_184096565021911_1037371396361086.json
2181_114517875225866_1463683516975955.json
1049_440106476051475_1246441708751277.json
0931_135665053303678_553000608236785.json
1382_62317591679_10153841493231680.json
1732_62317591679_10153860332881680.json
0084_184096565021911_1037782196320006.json
1053_440106476051475_1246565388738909.json
2133_389658314427637_1540826605977463.json
1034_135665053303678_554840758052770.json
0332_146422995398181_1456923867681414.json
1153_346937065399354_1245663435526708.json
0535_219367258105115_1285544454820718.json
0815_135665053303678_551845471685632.json
0716_219367258105115_129130

1023_135665053303678_554597891410390.json
1236_346937065399354_1248473691912349.json
1045_440106476051475_1246207405441374.json
1990_389658314427637_1535347349858722.json
0127_184096565021911_1041085625989663.json
1890_62317591679_10153864344406680.json
0631_219367258105115_1289922197716277.json
0061_184096565021911_1037075549724004.json
2028_389658314427637_1536357929757664.json
1014_135665053303678_554510084752504.json
0610_219367258105115_1287046594670504.json
0846_135665053303678_552216481648531.json
1137_440106476051475_1253393644722750.json
1743_62317591679_10153860728181680.json
1145_440106476051475_1253659874696127.json
0094_184096565021911_1038000319631527.json
0190_184096565021911_1042165485881677.json
2105_389658314427637_1540384676021656.json
0020_184096565021911_1035619779869581.json
0046_184096565021911_1036370659794493.json
2148_389658314427637_1541316919261765.json
1994_389658314427637_1535389693187821.json
1875_62317591679_10153863827486680.json
0362_219367258105115_12

0407_219367258105115_1283015921740238.json
1223_346937065399354_1248606598565725.json
1515_62317591679_10153846438271680.json
1181_346937065399354_1246629568763428.json
1352_346937065399354_1254206008005784.json
0412_219367258105115_1283143941727436.json
1435_62317591679_10153843648936680.json
1949_389658314427637_1534333036626820.json
0420_219367258105115_1283316031710227.json
0248_146422995398181_1449551611751973.json
1684_62317591679_10153852873576680.json
0674_219367258105115_1290542204320943.json
1121_440106476051475_1252425721486209.json
0322_146422995398181_1456237357750065.json
1052_440106476051475_1246491382079643.json
0989_135665053303678_554325304770982.json
0518_219367258105115_1285250141516816.json
0944_135665053303678_553089264894586.json
1803_62317591679_10153861687866680.json
0014_184096565021911_1035527713212121.json
1017_135665053303678_554510524752460.json
0584_219367258105115_1286579191383911.json
1086_440106476051475_1248280588567389.json
1192_346937065399354_12468

1941_389658314427637_1534159839977473.json
1037_440106476051475_1245659788829469.json
1983_389658314427637_1535238946536229.json
1642_62317591679_10153851263791680.json
050_184096565021911_1036474553117437.json
2216_114517875225866_1465110096833297.json
0266_146422995398181_1450628841644250.json
2118_389658314427637_1540644545995669.json
1660_62317591679_10153851994871680.json
0626_219367258105115_1289795571062273.json
0914_135665053303678_552600701610109.json
1606_62317591679_10153849865081680.json
2065_389658314427637_1537296322997158.json
0562_219367258105115_1285912444783919.json
0307_146422995398181_1455558234484644.json
2137_389658314427637_1541143495945774.json
1240_346937065399354_1248608988565486.json
0559_219367258105115_1285864978121999.json
1123_440106476051475_1252426544819460.json
1341_346937065399354_1253904648035920.json
0024_184096565021911_1035679109863648.json
1152_346937065399354_1245652818861103.json
0191_184096565021911_1042171789214380.json
1383_62317591679_10153

In [6]:
for fb in facebook_posts:
    print(fb)
    print(facebook_posts[fb])
    print("\n")

1784_62317591679_10153861481676680.json
Here's what the candidates got right and wrong.


2280_114517875225866_1472557792755194.json
Nailed it.


0354_219367258105115_1281841091857721.json
We're in White Plains, New York, where Democratic presidential nominee Hillary Clinton is holding a press conference.


1752_62317591679_10153861035666680.json
The Saudis are leaning on U.S. companies to pressure Congress to kill the measure allowing victims of the Sept. 11, 2001 attacks to sue the kingdom.


0705_219367258105115_1291002827608214.json
Donald J. Trump's campaign manager, Kellyanne Conway, says Trump believes that "climate change is naturally occurring." http://cnn.it/2d3g18W


1975_389658314427637_1535093403217450.json
The hatred in America right now is about to blow-up in our faces. We are at war.


1555_62317591679_10153847662691680.json
The "Bartlet for America" crew is with Hillary Clinton.


1294_346937065399354_1252978291461889.json
EXACTLY!

Image by Occupy Democrats, LIKE our 



0278_146422995398181_1451126608261140.json



2166_114517875225866_1462687457075561.json
Sports stadiums are a HUGE ripoff to US taxpayers.


1711_62317591679_10153859552511680.json
Inside today's POLITICO Playbook: New polls show that the race between Hillary Clinton and Donald J. Trump is virtually tied on debate day.


1424_62317591679_10153843180546680.json
The conflicting accounts came after Donald Trump on Friday accused Hillary Clinton of starting what he called “the ‘birther’ controversy."


1954_389658314427637_1534385633288227.json
Do you live in Arizona? Have you ever changed a baby's diaper? You're a felon. FYI.


0899_135665053303678_552600548276791.json
What Obama just said in front of the entire U.N. is disgraceful. He needs to go NOW!


0804_135665053303678_551830561687123.json
www.EagleRising.com


0272_146422995398181_1450853194955148.json
Now we just need a Democratic Congress.


1098_440106476051475_1249041701824611.json
Hillary Clinton's campaign was just caught 

0383_219367258105115_1282437811798049.json
Donald J. Trump points to how Israel uses profiling and how they have "done an unbelievable job."


1690_62317591679_10153853130336680.json
"After further discussion with community leaders, we have decided to postpone Sunday's trip as not to impact the City's resources," Clinton communications director Jennifer Palmieri wrote in a statement.


0708_219367258105115_1291111790930651.json
"I think Donald just criticized me for preparing for this debate," Hillary Clinton says during 2016's first US presidential debate. "And yes, I did. And you know what else I prepared for? I prepared to be President. And that is a good thing."


0696_219367258105115_1290804370961393.json
"And they also had, gave me a defective mic. Did you notice that? My mic was defective within the room," Donald J. Trump tells a group of reporters after the debate.


1543_62317591679_10153847347491680.json
Donald J. Trump: "I see what’s going on in Chicago, I think stop-and-fri

0974_135665053303678_554123541457825.json
Telling it like it is!


0220_146422995398181_1447689015271566.json
When even George W. and his myriad international failures couldn't budge this tradition, you know that the entire world considers Trump to be a true and serious problem.


1162_346937065399354_1245867855506266.json
This is an outrage.


0950_135665053303678_553200771550102.json
FAIL: Dem Congressional Candidate Calls Hillary 'Honest'...Audience Boos and Laughs [VIDEO]


0548_219367258105115_1285738474801316.json
"I can tell you that the vetting is tough," he adds. "It is very, very stringent."


1716_62317591679_10153859704106680.json
Donald J. Trump's campaign manager: "You saw — and others were talking about it in the earlier segment — that this weekend was spent by editorial writers and people on Twitter and elsewhere really just trying to undercut Donald Trump before the debate."


1283_346937065399354_1252884128137972.json
Please!


0147_184096565021911_1041422675955958.js

Congressional sources tell CNN that Donald J. Trump's late effort to engage on this issue could throw the last-minute negotiations into flux.


0039_184096565021911_1036218169809742.json



1504_62317591679_10153846051046680.json
Here's a preview of what she'll say.


1757_62317591679_10153861274306680.json
For the first time ever, a televised general election presidential debate won't include talk about the color of two men's ties.


1597_62317591679_10153849518081680.json
The director of the web series said Hillary Clinton really liked the episode with President Obama.


1173_346937065399354_1246088545484197.json
You read that right.


0951_135665053303678_553212381548941.json
ANYONE WHO Still Supports Hillary After They See This Video Should Forfeit Their Right To Vote

Do you agree?


0816_135665053303678_551845505018962.json
WATCH: It's well documented that Hillary will lie and cheat to further her personal wealth and political power but this video just might be the final DEVASTAT

2165_114517875225866_1462684057075901.json
This is how to save a river.


0063_184096565021911_1037112079720351.json
Recent polling suggests that when it comes to winning the youth vote (a key demographic for President Obama) perhaps Hillary Clinton’s greatest challenge is not Donald J. Trump, whom Bernie Sanders has been harshly critiquing on the trail, but other third-party candidates.


1794_62317591679_10153861561656680.json
Hillary Clinton: "This is a man who has called women pigs, slobs and dogs."


0142_184096565021911_1041316092633283.json
Take a look back at some of Hillary Clinton's most memorable moments from the Democratic primary #debates: abcn.ws/2d1kj0A


0336_146422995398181_1457164467657354.json
Dean, a physician, lays down a pretty good case for why we should be asking this question.


0355_219367258105115_1281841601857670.json
Hillary Clinton and Donald J. Trump are set to meet with Egyptian President Abdel Fattah el-Sisi at the United National General Assembly.


03

In [7]:
len(facebook_posts)

2291

In [9]:
sorted(facebook_posts, key=facebook_posts.__getitem__)

['0176_184096565021911_1041829352581957.json',
 '0090_184096565021911_1037889882975904.json',
 '0136_184096565021911_1041224572642435.json',
 '2266_114517875225866_1471951622815811.json',
 '2085_389658314427637_1539757599417697.json',
 '1207_346937065399354_1247725991987119.json',
 '0279_146422995398181_1451202248253576.json',
 '1281_346937065399354_1252873851472333.json',
 '1297_346937065399354_1252994151460303.json',
 '2241_114517875225866_1467439823266991.json',
 '2129_389658314427637_1540814195978704.json',
 '2273_114517875225866_1472332886111018.json',
 '2259_114517875225866_1471196759557964.json',
 '0278_146422995398181_1451126608261140.json',
 '0688_219367258105115_1290743144300849.json',
 '0028_184096565021911_1035738123191080.json',
 '2174_114517875225866_1462891757055131.json',
 '0483_219367258105115_1284522954922868.json',
 '0657_219367258105115_1290342521007578.json',
 '2084_389658314427637_1539756122751178.json',
 '1218_346937065399354_1248417465251305.json',
 '2208_114517

In [73]:
facebook_posts_list = list(facebook_posts.values())
facebook_posts_keys_list = list(facebook_posts.keys())
facebook_posts_list

["Here's what the candidates got right and wrong.",
 'Nailed it.',
 "We're in White Plains, New York, where Democratic presidential nominee Hillary Clinton is holding a press conference.",
 'The Saudis are leaning on U.S. companies to pressure Congress to kill the measure allowing victims of the Sept. 11, 2001 attacks to sue the kingdom.',
 'Donald J. Trump\'s campaign manager, Kellyanne Conway, says Trump believes that "climate change is naturally occurring." http://cnn.it/2d3g18W',
 'The hatred in America right now is about to blow-up in our faces. We are at war.',
 'The "Bartlet for America" crew is with Hillary Clinton.',
 'EXACTLY!\n\nImage by Occupy Democrats, LIKE our page for more!',
 'Why Aren’t the Out-of-State Rioters Facing FEDERAL Charges?',
 'Donald J. Trump reacts to the racial unrest in Charlotte saying, there is "a lack of spirit between the white and the black."',
 'Screening at airports does not just entail putting everyone through a metal detector.',
 'BUSTED! In a 

In [15]:
def preprocess(text):
    text = text.lower()
    doc = word_tokenize(text)
    doc = [word for word in doc if word not in stop_words]
    doc = [word for word in doc if word.isalpha()] #restricts string to alphabetic characters only
    return doc

In [24]:
#def filter_docs(corpus, texts, labels, condition_on_doc):
def filter_docs(corpus, texts, condition_on_doc):
    """
    Filter corpus, texts and labels given the function condition_on_doc which takes
    a doc.
    The document doc is kept if condition_on_doc(doc) is true.
    """
    number_of_docs = len(corpus)

    #if texts is not None:
    #    texts = [text for (text, doc) in zip(texts, corpus)
    #             if condition_on_doc(doc)]

    #labels = [i for (i, doc) in zip(labels, corpus) if condition_on_doc(doc)]
    #corpus = [doc for doc in corpus if condition_on_doc(doc)]
    
    if texts is not None:
        for i in range(number_of_docs):
            doc = corpus[i]
            if condition_on_doc(doc) is False or doc == 'nan':
                texts[i] = ''
                corpus[i] = []
                

    print("{} docs removed".format(number_of_docs - len(corpus)))

    return (corpus, texts)

In [21]:
instagram_corpus = [preprocess(str(text)) for text in instagram_text_list]
len(instagram_corpus)

101

In [68]:
facebook_corpus = [preprocess(str(text)) for text in facebook_posts_list]
len(facebook_corpus)

2291

In [37]:
# Remove OOV words and documents with no words in model dictionary
def document_vector(word2vec_model, doc):
    # remove out-of-vocabulary words
    doc = [word for word in doc if word in word2vec_model.vocab]
    return np.mean(word2vec_model[doc], axis=0)

In [38]:
def has_vector_representation(word2vec_model, doc):
    """check if at least one word of the document is in the
    word2vec dictionary"""
    return not all(word not in word2vec_model.vocab for word in doc)

In [39]:
instagram_corpus, instagram_text_list = filter_docs(instagram_corpus, instagram_text_list, lambda doc: has_vector_representation(model, doc))

0 docs removed


In [40]:
facebook_corpus, facebook_posts_list = filter_docs(facebook_corpus, facebook_posts_list, lambda doc: has_vector_representation(model, doc))

0 docs removed


In [34]:
instagram_word2vec =[]
for doc in instagram_corpus: #look up each doc in model
    #print("i: ", i)
    #i = i + 1
    if len(doc) == 0:
        instagram_word2vec.append(np.zeros((300,)))
    else:
        instagram_word2vec.append(document_vector(model, doc))

In [41]:
facebook_word2vec =[]
for doc in facebook_corpus: #look up each doc in model
    #print("i: ", i)
    #i = i + 1
    if len(doc) == 0:
        facebook_word2vec.append(np.zeros((300,)))
    else:
        facebook_word2vec.append(document_vector(model, doc))

In [43]:
facebook_corpus[0]

['candidates', 'got', 'right', 'wrong']

In [45]:
facebook_word2vec[0].shape

(300,)

In [55]:
similarity_summary_dict = {}
i = 0
for ins_w2c in instagram_word2vec:
    sim_list = []
    for fb_w2c in facebook_word2vec:
        #print(ins_w2c)
        x1i = ins_w2c.reshape(1,300)
        x2i = fb_w2c.reshape(1,300)
        sim = cosine_similarity(x1i, x2i)
        sim_list.append(sim[0,0])
        #print(sim[0,0])
    similarity_summary_dict[i] = sim_list
    i = i + 1
    print("%s is finished\n" % str(i))

1 is finished

2 is finished

3 is finished

4 is finished

5 is finished

6 is finished

7 is finished

8 is finished

9 is finished

10 is finished

11 is finished

12 is finished

13 is finished

14 is finished

15 is finished

16 is finished

17 is finished

18 is finished

19 is finished

20 is finished

21 is finished

22 is finished

23 is finished

24 is finished

25 is finished

26 is finished

27 is finished

28 is finished

29 is finished

30 is finished

31 is finished

32 is finished

33 is finished

34 is finished

35 is finished

36 is finished

37 is finished

38 is finished

39 is finished

40 is finished

41 is finished

42 is finished

43 is finished

44 is finished

45 is finished

46 is finished

47 is finished

48 is finished

49 is finished

50 is finished

51 is finished

52 is finished

53 is finished

54 is finished

55 is finished

56 is finished

57 is finished

58 is finished

59 is finished

60 is finished

61 is finished

62 is finished

63 is finished

6

In [57]:
len(similarity_summary_dict[0])

2291

In [58]:
similarity_summary_dict[0]

[0.15245567,
 0.073867403,
 0.53841072,
 0.22823198,
 0.3336668,
 0.4025493,
 0.62476194,
 0.3970921,
 0.11239488,
 0.44376141,
 0.18686299,
 0.33485478,
 0.19284731,
 0.64937919,
 0.34427479,
 0.65371668,
 0.10536022,
 0.27893943,
 0.061434153,
 0.42864785,
 0.28396857,
 0.37291884,
 0.45982647,
 0.36793125,
 0.096554913,
 0.36123726,
 0.26924312,
 0.36696956,
 0.44052002,
 0.37342003,
 0.53459442,
 0.37425077,
 0.32542947,
 0.50407046,
 0.24402842,
 0.48933765,
 0.36379382,
 0.19534704,
 0.46156585,
 0.20862304,
 0.27469319,
 0.38942525,
 0.27345333,
 0.28908721,
 0.0,
 0.09619534,
 0.3192291,
 0.46896836,
 0.2681919,
 0.15838058,
 0.18184914,
 0.22204016,
 0.47240669,
 0.2059634,
 0.63879371,
 0.65135801,
 0.30459729,
 0.37235591,
 0.33194584,
 0.39797005,
 0.35868785,
 0.12885201,
 0.36414927,
 0.56816047,
 0.0,
 0.60247356,
 0.39259851,
 0.16123839,
 0.29633173,
 0.59198707,
 0.0,
 0.57270622,
 0.2006136,
 0.26234451,
 0.4227297,
 0.17449333,
 0.41694316,
 0.44593364,
 0.61234063,

In [83]:
file = open('summary_word2vec.csv', 'w')
i = 0
head = "instagram_idx,instragram_comment,facebook_json,facebook_message,similarity\n"
file.write(head)

for sim in similarity_summary_dict.values():
    
    print(i, ': \n')
    sim_max = max(sim)
    sim_idx = sim.index(max(sim))
    
    instagram_comment = instagram_text_list[i].replace(",", " ").replace("\n"," ")
    print('instagram comment: \n')
    print(instagram_comment)
    print(instagram_corpus[i])
    
    facebook_post = facebook_posts_list[sim_idx].replace(",", " ").replace("\n"," ")
    print('facebook message: \n')
    print(facebook_posts_keys_list[sim_idx])
    print(facebook_post)
    print(facebook_corpus[sim_idx])
    print(sim_max, sim_idx)
    print('-------------------------\n')
    
    data = "%d,%s,%s,%s,%f\n" % (i,instagram_comment,facebook_posts_keys_list[sim_idx],facebook_post,sim_max)
    file.write(data)
    i = i + 1
    
file.close()

0 : 

instagram comment: 

Catherine Duchess of Cambridge and Prince William Duke of Cambridge visit the Great Bear Rainforest in Bella Bella Canada. PHOTO: Sam Hussein / Getty Images. #Royals #RoyalFamily #PrinceWilliam #DuchessCatherine #Canada
['catherine', 'duchess', 'cambridge', 'prince', 'william', 'duke', 'cambridge', 'visit', 'great', 'bear', 'rainforest', 'bella', 'bella', 'canada', 'photo', 'sam', 'hussein', 'getty', 'images', 'royals', 'royalfamily', 'princewilliam', 'duchesscatherine', 'canada']
facebook message: 

0828_135665053303678_552093584994154.json
Here's a FLASHBACK for CNN  Obama  Sheila Jackson Lee & Hillary. They said it. Enjoy.
['flashback', 'cnn', 'obama', 'sheila', 'jackson', 'lee', 'hillary', 'said', 'enjoy']
0.717124 1837
-------------------------

1 : 

instagram comment: 

The Presidential Debate ends on a handshake between Hillary Clinton and Donald Trump. #Debates
['presidential', 'debate', 'ends', 'handshake', 'hillary', 'clinton', 'donald', 'trump', '